In [1]:
import pandas as pd

from flags.lrh import FlagUnembeddingRepresentation
from flags.nlp.datasets import load_multilingual_question_dataset
from flags.utils.plotting import lineplot_and_save

/home/pedro/Documents/flag-representation-hypothesis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# parameters
MODEL_ID = "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4"
N = 1 << 9
STEPS = 1 << 4
BATCH_SIZE = 1 << 5

GUIDE = ("woman.n.01", "male.n.01")

X = "token index"
Y = "total projection"
HUE = "language"

LANG_ABLATION_K = 3

In [ ]:
fur = FlagUnembeddingRepresentation.from_model_id(MODEL_ID)
print("memory cost: ", fur.model.memory_footprint >> 20, "Mb")

In [ ]:
dataset = load_multilingual_question_dataset(fur.data.languages).iloc[:N]
samples, langs = dataset.shape
print("Loaded dataset with", samples, "samples and", langs, "languages")

In [ ]:
def gen_guided_probe(inputs, k):
    _, probe = fur.quick_generate_with_topk_guide(
        inputs,
        guide=GUIDE,
        k=k,
        steps=STEPS,
        batch_size=BATCH_SIZE,
        min_lemmas_per_synset=3,
        max_token_count=3,
    )
    return pd.DataFrame(probe).melt(var_name=X, value_name=Y)

In [ ]:
df = pd.concat(
    [
        gen_guided_probe(dataset[lang].tolist(), k=LANG_ABLATION_K).assign(
            **{HUE: lang}
        )
        for lang in dataset.columns
    ]
)

lineplot_and_save("09_guided_generation_lang_comparison", df, x=X, y=Y, hue=HUE)